In [17]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

from skimage.transform import resize
import json
import requests

In [25]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data

def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_mean, img_std, img_size): 
    img = img / 255
#     grey_image = (img - img_mean)/img_std
    grey_image = img
#     grey_image = grey_image.reshape(1, 224,224)
    grey_image = resize(grey_image,(224,224) )
    proc_img = np.zeros((224,224,3))
    proc_img[:, :, 0] = grey_image
    proc_img[:, :, 1] = grey_image
    proc_img[:, :, 2] = grey_image
    
    proc_img = np.resize(proc_img, img_size)
   
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    """
    :param model_path: the path of the model
    :para,weight_path: the path of the weights model
    """
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()    
    model = keras.models.model_from_json(loaded_model_json)
    model.load_weights(weight_path)

    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo    
    pred_Y = model.predict(img)
#     print(pred_Y)
    prediction = np.where(pred_Y>thresh, "pneumonia", "no finding")
    
    return prediction 

In [30]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']
model_path = "my_model.json"

weight_path = "xray_class_vgg_v3_my_model.best.hdf5"#path to saved best weights

IMG_SIZE = (1, 224,224, 3) # This might be different if you did not use vgg16

# loads the mean image value they used during training preprocessing
# loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)
thresh = 0.31 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    img_mean = np.mean(img)
    img_std = np.std(img)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
[[0.08646312]]
[['no finding']]
Load file test2.dcm ...
[[0.20708096]]
[['no finding']]
Load file test3.dcm ...
[[0.2674405]]
[['no finding']]
Load file test4.dcm ...
[[0.08646312]]
[['no finding']]
Load file test5.dcm ...
[[0.08646312]]
[['no finding']]
Load file test6.dcm ...
[[0.08646312]]
[['no finding']]


In [28]:
dfpatienst = get_df_dicom("test", 1, 7)

In [29]:
dfpatienst

,patient_id,patient_age,patient_sex,modality,type_finding,rows,columns,pixel_array
0,2,81,M,DX,No Finding,1024,1024,"[[199, 175, 152, 133, 124, 118, 113, 111, 110,..."
0,1,58,M,DX,Cardiomegaly,1024,1024,"[[202, 199, 195, 193, 195, 194, 193, 192, 184,..."
0,61,77,M,DX,Effusion,1024,1024,"[[142, 142, 143, 141, 143, 140, 140, 136, 137,..."
0,2,81,M,DX,No Finding,1024,1024,"[[199, 175, 152, 133, 124, 118, 113, 111, 110,..."
0,2,81,M,CT,No Finding,1024,1024,"[[199, 175, 152, 133, 124, 118, 113, 111, 110,..."
0,2,81,M,DX,No Finding,1024,1024,"[[199, 175, 152, 133, 124, 118, 113, 111, 110,..."


----

In [27]:
def get_df_dicom(label_image, initial, final):
    
    df_list = []
    
    for i in range(initial, final):
        dcm_obj = pydicom.dcmread("{}{}.dcm".format(label_image, i))
        df_tmp = pd.DataFrame({'patient_id':[dcm_obj.PatientID],
                     'patient_age':[dcm_obj.PatientAge],
                               'patient_sex':[dcm_obj.PatientSex],
                               'modality':[dcm_obj.Modality],
                               'type_finding':[dcm_obj.StudyDescription],
                               'rows':[dcm_obj.Rows],
                               'columns':[dcm_obj.Columns],
                               'pixel_array':[dcm_obj.pixel_array]
                              })
        df_list.append(df_tmp)
    dfpatients = pd.concat(df_list)
    
    return dfpatients